#### CSC 180 Intelligent Systems 

#### William Lorence, Ajaydeep Singh, 

#### California State University, Sacramento


# Project 1: Yelp Business Rating Prediction using Tensorflow


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "./yelp_dataset/"
save_path = "./models/"

df_review = pd.read_json('./yelp_dataset/yelp_academic_dataset_review.json', lines=True, nrows = 1000000)
df_business = pd.read_json('./yelp_dataset/yelp_academic_dataset_business.json', lines=True, nrows = 1000000)

df_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


Groups all reviews (text) by business_id and concatenates all the reviews for each business into a single string. This means that for each business, you'll get one entry where all its reviews are combined into one text entry.

df_ready_to_be_sent_to_sklearn converts the df_review_agg series into a **DataFrame**


In [21]:
df_review_agg = df_review.groupby('business_id')['text'].sum()
df_ready_to_be_sent_to_sklearn = pd.DataFrame({'business_id': df_review_agg.index, 'all_reviews': df_review_agg.values})

The below merges df_ready_to_be_sent_to_sklearn (which contains the concatenated reviews for each business) with df_business (which contains other information about each business such as name, category, location, etc).

df_review_business.shape = returns the shape of the resulting DataFrame (df_review_business), which tells you the number of rows and columns.

In [24]:
df_review_business = pd.merge(df_ready_to_be_sent_to_sklearn, df_business, on='business_id')
df_review_business.shape


(27095, 15)

In [23]:
df_review_business.head()

,business_id,all_reviews,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,--ZVrH2X2QXBFdCilbirsw,This place is sadly perm closed. I was hoping ...,Chris's Sandwich Shop,1531 W Wynnewood Rd,Ardmore,PA,19003,39.997299,-75.292207,4.5,32,0,"{'GoodForKids': 'True', 'RestaurantsAttire': '...","American (Traditional), Restaurants, Pizza, Sa...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
1,--_9CAxgfXZmoFdNIRrhHA,It's a huge place. There's P.O. boxes big enou...,USPS,3135 1st Ave N,St. Petersburg,FL,33730,27.772845,-82.676156,3.5,11,1,None,"Post Offices, Public Services & Government",None
2,--_lZuj_WCGnDG6n0emlRg,Good deals to be had on liquidated designer cl...,Ross Dress for Less,700 Haddonfield Berlin Rd,Voorhees,NJ,08043,39.850059,-74.979995,2.0,13,1,"{'BikeParking': 'False', 'BusinessAcceptsCredi...","Shopping, Women's Clothing, Discount Store, Ho...","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
3,--hF_3v1JmU9nlu4zfXJ8Q,Really excited to get healthier options on the...,Green District Salads - Monument Circle,28 Monument Cir,Indianapolis,IN,46204,39.767888,-86.158351,4.5,15,1,"{'Alcohol': 'u'beer_and_wine'', 'OutdoorSeatin...","Vegetarian, Restaurants, Wraps, Salad","{'Monday': '0:0-0:0', 'Tuesday': '11:0-16:0', ..."
4,--sXnWH9Xm6_NvIjyuA99w,Ich war das erste mal in Philadelphia und ich ...,Philadelphia,,Philadelphia,PA,,39.952584,-75.165222,4.0,29,1,{'GoodForKids': 'True'},"Public Services & Government, Local Flavor",None
